In [1]:
import configparser
import json
import requests
from bs4 import BeautifulSoup

In [2]:
config = configparser.ConfigParser()
config.read('.password', 'UTF-8')
 
username = config.get("settings", 'username')
password = config.get("settings", 'password')

In [3]:
session = requests.Session()

# くらしtepcoからでんき家計簿にログイン
param = {
        'ACCOUNTUID': username,
        'PASSWORD': password,
        'HIDEURL': '/ls/pf/ja/pco/mypage/redirect-sso.page?sitekbn=kakeibo',
        'LOGIN': 'EUAS_LOGIN',
        }

header = {
        'Referer': 'https://www.kakeibo.tepco.co.jp/dk/com/menu/'
        }

login = session.post(
        'https://www.kurashi.tepco.co.jp/kpf-login', data=param, headers=header)
login.encoding = login.apparent_encoding

In [4]:
# 月ごとデータへの移動リンクで発火するjs関数から、必要な情報を抜き出す
html = BeautifulSoup(login.text, "html.parser")
link_js = html.find(id="frame3").find(class_="box01 box firstBox").find("a")["onclick"]
#print(link_js)
key_officeCd = link_js.split("'")[3]
key_visitNum = link_js.split("'")[5]
key_display = link_js.split("'")[7]
post_to = link_js.split("'")[17]

#print(key_officeCd)
print("***my officeCd***")
#print(key_visitNum)
print("***my visitNum***")
print(key_display)
print(post_to)

***my officeCd***
***my visitNum***
amount
/dk/com/menu/goElectricUsageAmount


In [5]:
# 月ごとデータの表示ページ に移動
param = {
        'key.officeCd': key_officeCd,
        'key.visitNum': key_visitNum,
        'key.display': key_display
}

header = {
        'Referer':login.url
        }

data_page = session.post(
        'https://www.kakeibo.tepco.co.jp'+post_to, data=param, headers=header)
data_page.encoding = data_page.apparent_encoding

In [6]:
def getCDATA(text):
    ret = ""
    isInCDATAArea = False
    for l in text.split("\n"):
        l = l.strip()
        if (l == "// <![CDATA["):
            isInCDATAArea = True
        elif (l.strip() == "// ]]>"):
            isInCDATAArea = False
        
        if(isInCDATAArea):
            ret += l + "\n"
        
    return ret

#print(getCDATA(data_page.text))

In [7]:
d = {}
function = ""
for l in getCDATA(data_page.text).split("\n"):
    
    if(l.startswith("function")):
        function = l.split("function")[1].split("()")[0].strip()
        if(function.startswith("vbar")):
            d[function] = {}
        
    if(function.startswith("vbar")):
        if(l.startswith("var items = ")):
            d[function]["items"] = l.split("=")[-1]

        if(l.startswith("x:")):
            d[function]["x"] = json.loads(l.split(":")[-1])
        
        if(l.startswith(",y:")):
            d[function]["y"] = json.loads(l.split(":")[-1])

In [ ]:
for f in d:
    print(f)
    for k in d[f]:
        print("\t", k, "\t", d[f][k])

In [8]:
with open("monthly.json", "w") as f:
    f.write(json.dumps(d))